In [13]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle

In [2]:
# File path
FILE_PATH = './Dataset/EURUSD/EURUSD_M30_features+label_v.2.1.csv'
# Hyperparameters
seq_length = 15
num_layers = 2
num_heads = 4
batch_size = 1024
epochs = 50
dropout = 0.4
learning_rate = 0.005

In [3]:
# Read the dataset
data = pd.read_csv(FILE_PATH)

In [4]:
# Selecting features and labels
features = ['Close', 'SMA200', 'SMA50', 'RSI14']
label_column = 'signal'
data = data[features + [label_column]]

In [5]:
# Normalize the data and round to 4 decimal places
scaler = MinMaxScaler()
data[features] = scaler.fit_transform(data[features])
data = data.round(4)

In [6]:
# Handle labels
data[label_column] = data[label_column].apply(lambda x: int(x) if x in [1, 2, 3] else 3)

In [7]:
# Split the data
X = data[features].values
y = data[label_column].values - 1  # Adjusting labels for 0-based indexing

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Converting to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [9]:
# Transformer model definition
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_heads, num_layers, seq_length, dropout, num_classes):
        super(TransformerModel, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dropout=dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(
            self.encoder_layer, num_layers=num_layers
        )
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        x = self.transformer_encoder(x)
        x = self.fc(x[:, -1, :])  # Taking the last token's output
        return x

In [14]:
# Save the processed data to a pickle file
processed_data = {
    'X_train': X_train,
    'y_train': y_train,
    'X_test': X_test,
    'y_test': y_test
}

pickle_path = "./Dataset/EURUSD/EURUSD_M30_features+label_v.2.2.pkl"
with open(pickle_path, 'wb') as file:
    pickle.dump(processed_data, file)

In [10]:
# Instantiate the model
input_dim = len(features)
num_classes = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = TransformerModel(input_dim, num_heads, num_layers, seq_length, dropout, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


c:\Program Files\Python312\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [11]:
# Training loop
def train_model(model, X_train, y_train, batch_size, epochs, criterion, optimizer):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for i in range(0, len(X_train) - seq_length, batch_size):
            optimizer.zero_grad()
            batch_x = [X_train[j:j+seq_length] for j in range(i, min(i+batch_size, len(X_train)-seq_length))]
            batch_y = y_train[i:i+len(batch_x)]
            
            batch_x = torch.stack(batch_x).to(device)
            batch_y = batch_y.to(device)
            
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

train_model(model, X_train, y_train, batch_size, epochs, criterion, optimizer)


Epoch 1/50, Loss: 164.1566
Epoch 2/50, Loss: 163.0373
Epoch 3/50, Loss: 163.0244
Epoch 4/50, Loss: 162.9927
Epoch 5/50, Loss: 163.0084
Epoch 6/50, Loss: 162.9855
Epoch 7/50, Loss: 162.9963
Epoch 8/50, Loss: 162.9864
Epoch 9/50, Loss: 163.0043
Epoch 10/50, Loss: 162.9905
Epoch 11/50, Loss: 162.9811
Epoch 12/50, Loss: 162.9730
Epoch 13/50, Loss: 162.9767
Epoch 14/50, Loss: 162.9603
Epoch 15/50, Loss: 162.9711
Epoch 16/50, Loss: 162.9669
Epoch 17/50, Loss: 162.9632
Epoch 18/50, Loss: 162.9560
Epoch 19/50, Loss: 162.9596
Epoch 20/50, Loss: 162.9629
Epoch 21/50, Loss: 162.9627
Epoch 22/50, Loss: 162.9566
Epoch 23/50, Loss: 162.9573
Epoch 24/50, Loss: 162.9558
Epoch 25/50, Loss: 162.9549
Epoch 26/50, Loss: 162.9459
Epoch 27/50, Loss: 162.9511
Epoch 28/50, Loss: 162.9422
Epoch 29/50, Loss: 162.9461
Epoch 30/50, Loss: 162.9457
Epoch 31/50, Loss: 162.9462
Epoch 32/50, Loss: 162.9442
Epoch 33/50, Loss: 162.9437
Epoch 34/50, Loss: 162.9391
Epoch 35/50, Loss: 162.9437
Epoch 36/50, Loss: 162.9394
E